In [0]:
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

In [0]:
import os, sys

# Allow load project as module
sys.path.insert(0, '../../../../python')

In [0]:
%matplotlib inline

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Open stellar grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.wave_lim = [3000, 9000]
grid.preload_arrays = False
grid.load(fn, format='h5')

# Resampling

In [0]:
from pfs.ga.pfsspec.core.obsmod.psf import VelocityDispersion
from pfs.ga.pfsspec.core.obsmod.resampling import *

In [0]:
model = grid.get_model(M_H=0, T_eff=4500, log_g=1, a_M=0, )
model.wave.shape, model.flux.shape, model.wave[0], model.wave[-1]

psf = VelocityDispersion(vdisp=1, reuse_kernel=True)
s = psf.get_optimal_size(model.wave)

model.convolve_psf(psf, size=s)

In [0]:
model.wave[[0, 1, 2, -3, -2, -1]], np.diff(model.wave)[[0, 1, -2, -1]]

In [0]:
wave_edges = FluxConservingResampler().find_wave_edges(model.wave)
wave_edges[[0, 1, -2, -1]], np.diff(wave_edges)[[0, 1, -2, -1]]

In [0]:
plt.plot(model.wave, np.diff(wave_edges))

In [0]:
# Target wave

#wave_edges = np.linspace(8000, 8500, 501)
#wave_edges = np.linspace(3001, 3021, 21)
wave_edges = np.linspace(8200, 8215, 16)

wave = 0.5 * (wave_edges[1:] + wave_edges[:-1])

res = Interp1dResampler(kind='linear')
model_interp1d = model.copy()
model_interp1d.apply_resampler(res, wave, wave_edges)

res = FluxConservingResampler()
model_cumsum = model.copy()
model_cumsum.apply_resampler(res, wave, wave_edges)

res = PysynphotResampler()
model_synphot = model.copy()
model_synphot.apply_resampler(res, wave, wave_edges)

res = SpecutilsResampler()
model_specutils = model.copy()
model_specutils.apply_resampler(res, wave, wave_edges)

In [0]:
f, axs = plt.subplots(2, 1, figsize=(5, 5), dpi=300)

ax = axs[0]
ax.step(model.wave, model.flux, where='mid', lw=0.3, label='model')
ax.step(model_interp1d.wave, model_interp1d.flux, where='mid', lw=0.3, label='interp1d')
ax.step(model_cumsum.wave, model_cumsum.flux, where='mid', lw=0.3, label='cumsum trick')
ax.step(model_synphot.wave, model_synphot.flux, where='mid', lw=0.3, label='synphot')
ax.step(model_specutils.wave, model_specutils.flux, where='mid', lw=0.5, label='specutils')

ax.legend(ncol=3)

ax.set_xlim(8200, 8215)
ax.set_ylim(0.5e14, 2.0e14)

#ax.set_xlim(3000, 3015)
#ax.set_ylim(0, 0.35e13)

ax = axs[1]

ax.step(model_specutils.wave, (model_cumsum.flux - model_specutils.flux) /  model_cumsum.flux, where='mid', lw=0.5, label='specutils')

# Illustration of the method

In [0]:
# Generate some random spectrum

wave_edges = np.linspace(0, 10, 41)
wave = (wave_edges[1:] + wave_edges[:-1]) / 2
flux = np.random.randint(0, 5, size=wave.shape).astype(np.float64)

nwave_edges = np.linspace(1.1, 8.7, 8)
nwave = (nwave_edges[1:] + nwave_edges[:-1]) / 2

In [0]:
# Use the actual model

wave_edges = FluxConservingResampler().find_wave_edges(model.wave)
wave = model.wave
flux = model.flux

# print(model.wave.shape, wave_edges.shape)
# #idx = np.digitize([8202, 8206], wave_edges)
# #idx = np.digitize([8200, 8215], wave_edges)
# #idx = np.digitize([3000, 3015], wave_edges)
# idx = np.digitize([3000, 8300], wave_edges)
# print(idx)
# wave_edges = wave_edges[idx[0]:idx[1]]
# wave = model.wave[idx[0]+1:idx[1]]
# flux = model.flux[idx[0]+1:idx[1]]

#nwave_edges = np.linspace(8202.5, 8205.5, 8)
#nwave_edges = np.linspace(8200, 8215, 16)
#nwave_edges = np.linspace(3001, 3021, 21)
nwave_edges = np.linspace(3001, 8300, 5301)
nwave = (nwave_edges[1:] + nwave_edges[:-1]) / 2

wave.shape, wave_edges.shape

In [0]:
from scipy.interpolate import interp1d

def steps(wave_edges, flux):
    we = np.stack([wave_edges[:-1], wave_edges[1:]], axis=-1).flatten()
    fl = np.stack(2 * [flux], axis=-1).flatten()
    return we, fl

if True:
    # cumsum resampling
    bs = wave_edges[1:] - wave_edges[:-1]       # bin size
    cs = np.empty_like(wave_edges)
    cs[0] = 0
    cs[1:] = np.cumsum(flux * bs)

    # interpolate to the new grid
    ip = interp1d(wave_edges, cs, bounds_error=False, fill_value=(0, cs[-1]))
    nflux = np.diff(ip(nwave_edges)) / np.diff(nwave_edges)

if False:
    res = FluxConservingResampler()
    res.init(nwave, nwave_edges)
    nflux, _ = res.resample_value(wave, wave_edges, flux)
    res.reset()

if False:
    # specutils resamplings
    from astropy import units as u
    from specutils import Spectrum1D
    from specutils.manipulation import FluxConservingResampler as specutils_fcr

    s = Spectrum1D(spectral_axis=wave * u.AA, flux=flux * u.Unit('erg cm-2 s-1 AA-1'))
    ns = specutils_fcr()(s, nwave * u.AA)
    nflux_su = ns.flux.value

In [0]:
f, axs = plt.subplots(2, 1, figsize=(6, 6), dpi=240)

ax = axs[0]
ax.plot(*steps(wave_edges, flux), '.-')
ax.plot(*steps(nwave_edges, nflux), '.-')
#ax.plot(*steps(nwave_edges, nflux_su), '.--')
#ax.set_ylim(0.5e14, 2.0e14)
#ax.set_ylim(0.5e14, 2.0e14)
ax.set_ylabel(r'$F_\lambda$')

### ### ### ### ###

ax = axs[1]
#ax.step(wave_edges, cs, where='mid')
ax.plot(wave_edges, cs, '.-')
ax.plot(nwave_edges, ip(nwave_edges), '.')
ax.set_ylabel(r'cummulative $F$')

for ax in axs:
    ax.grid()

    ax.set_xlim(-1, 11)
    #ax.set_xlim(8200, 8215)
    #ax.set_xlim(3000, 3015)